# 🧪 Testing with ScalaTest

**Phase 2 (Intermediate) - Module 4 of 6**

**Estimated time**: 90-120 minutes

**Prerequisites**: [03_Futures_AsyncProgramming.ipynb](03_Futures_AsyncProgramming.ipynb)

## 🎯 Learning Goals

- Master unit testing with ScalaTest
- Write different types of tests (Unit, Integration, Property)
- Use test fixtures and lifecycle methods
- Implement test-driven development (TDD)
- Handle async testing with Futures
- Create comprehensive test suites

---

## 📋 Table of Contents

1. [Testing Fundamentals](#fundamentals)
2. [Unit Testing](#unit)
3. [Matchers and Assertions](#matchers)
4. [Test Fixtures](#fixtures)
5. [Async Testing](#async)
6. [Property-Based Testing](#property)
7. [TDD Example](#tdd)
8. [Exercises](#exercises)
9. [What Next](#next)

## 💡 Why Testing Matters?

**The Problem:**
- Code changes can introduce bugs
- Manual testing is slow and unreliable
- Refactoring becomes scary without tests
- Teams waste time debugging

**The Solution: Automated Testing**
- **Fast feedback**: Tests run in seconds
- **Confidence**: Refactor without fear
- **Documentation**: Tests document expected behavior
- **Design**: Tests drive better code design
- **Regression**: Catch bugs before they reach production

**ScalaTest Styles:**
- **FunSuite**: Simple, readable tests
- **FlatSpec**: BDD-style specifications
- **WordSpec**: More readable BDD
- **FeatureSpec**: Acceptance testing

---

## 🧪 Testing Fundamentals

Setting up your testing environment with ScalaTest.

In [ ]:
// In a real Scala project, add to build.sbt:
// libraryDependencies += "org.scalatest" %% "scalatest" % "3.2.15" % Test

// For this notebook, we'll simulate testing concepts
println("=== Testing Setup ===")
println("In a real project:")
println("1. Add ScalaTest to build.sbt")
println("2. Tests go in src/test/scala/")
println("3. Run with: sbt test")
println("4. Individual test: sbt \"testOnly YourTestClass\"")
println()

## 🧰 Unit Testing Basics

Writing your first unit tests with FunSuite.

In [ ]:
// Example class to test
class Calculator {
  def add(a: Int, b: Int): Int = a + b
  def multiply(x: Int, y: Int): Int = x * y
  def divide(dividend: Int, divisor: Int): Option[Int] = {
    if (divisor == 0) None else Some(dividend / divisor)
  }
  def isEven(n: Int): Boolean = n % 2 == 0
}

// Simulate FunSuite testing (in real project, extend FunSuite)
class CalculatorTest {
  println("=== Calculator Unit Tests ===")
  val calculator = new Calculator()
  
  def testAdd(): Unit = {
    val result = calculator.add(2, 3)
    if (result == 5) {
      println("✓ testAdd: PASSED")
    } else {
      println(s"✗ testAdd: FAILED (expected 5, got $result)")
    }
  }
  
  def testMultiply(): Unit = {
    val result = calculator.multiply(4, 7)
    if (result == 28) {
      println("✓ testMultiply: PASSED")
    } else {
      println(s"✗ testMultiply: FAILED (expected 28, got $result)")
    }
  }
  
  def testDivide(): Unit = {
    val validResult = calculator.divide(10, 2)
    val invalidResult = calculator.divide(10, 0)
    
    if (validResult == Some(5) && invalidResult == None) {
      println("✓ testDivide: PASSED")
    } else {
      println(s"✗ testDivide: FAILED (valid=$validResult, invalid=$invalidResult)")
    }
  }
  
  def run(): Unit = {
    testAdd()
    testMultiply()
    testDivide()
    println()
  }
}

// Run tests
val calculatorTests = new CalculatorTest()
calculatorTests.run()

## 🎯 Assertions and Matchers

Using ScalaTest matchers for expressive assertions.

In [ ]:
// Simulate ScalaTest matchers
object ShouldMatchers {
  implicit class ShouldWrapper[A](actual: A) {
    def shouldEqual(expected: A): Boolean = {
      if (actual == expected) {
        println(s"✓ Assertion PASSED: $actual equals $expected")
        true
      } else {
        println(s"✗ Assertion FAILED: expected $expected, got $actual")
        false
      }
    }
    
    def shouldBe(expected: A): Boolean = shouldEqual(expected)
    
    def shouldContain[B >: A](item: B): Boolean = {
      actual match {
        case seq: Seq[B] => 
          if (seq.contains(item)) {
            println(s"✓ Contains assertion PASSED: $actual contains $item")
            true
          } else {
            println(s"✗ Contains assertion FAILED: $actual doesn't contain $item")
            false
          }
        case _ => false
      }
    }
  }
}

import ShouldMatchers._

// Test with simulated matchers
println("=== Matcher Assertions ===")

val list = List(1, 2, 3, 4, 5)
val sum = list.sum

sum shouldEqual 15
sum shouldBe 15
list shouldContain 3

val calculator = new Calculator()
calculator.add(5, 3) shouldEqual 8
calculator.isEven(4) shouldBe true
calculator.divide(10, 2).get shouldEqual 5

println()

## 🏗️ Test Fixtures

Setting up test data and cleanup in a structured way.

In [ ]:
// Simulate test fixture with BeforeAndAfter
class StringProcessorFixture {
  println("=== Test Fixture Lifecycle ===")
  
  class StringProcessor {
    def reverse(s: String): String = s.reverse
    def toUpperCase(s: String): String = s.toUpperCase
    def length(s: String): Int = s.length
  }
  
  // Fixture methods
  def beforeEach(): Unit = {
    println("Setting up test fixture...")
  }

  def afterEach(): Unit = {
    println("Cleaning up test fixture...")
  }
  
  // Test methods
  def testReverse(): Unit = {
    beforeEach()
    try {
      val processor = new StringProcessor()
      val result = processor.reverse("hello")
      if (result == "olleh") {
        println("✓ testReverse: PASSED")
      } else {
        println(s"✗ testReverse: FAILED (got '$result')")
      }
    } finally {
      afterEach()
    }
  }
  
  def testUpperCase(): Unit = {
    beforeEach()
    try {
      val processor = new StringProcessor()
      val result = processor.toUpperCase("hello")
      if (result == "HELLO") {
        println("✓ testUpperCase: PASSED")
      } else {
        println(s"✗ testUpperCase: FAILED (got '$result')")
      }
    } finally {
      afterEach()
    }
  }
  
  def run(): Unit = {
    testReverse()
    println()
    testUpperCase()
    println()
  }
}

val fixtureTests = new StringProcessorFixture()
fixtureTests.run()

## ⏱️ Testing Async Code

Testing Futures and asynchronous operations.

In [ ]:
import scala.concurrent.{Future, Await}
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.duration._

// Service with async operations
class AsyncService {
  def fetchData(id: Int): Future[String] = Future {
    Thread.sleep(100) // Simulate network call
    s"Data for $id"
  }
  
  def processData(data: String): Future[Int] = Future {
    Thread.sleep(50)
    data.length
  }
  
  def riskyOperation(): Future[String] = Future {
    Thread.sleep(25)
    if (scala.util.Random.nextBoolean()) {
      "Success"
    } else {
      throw new RuntimeException("Random failure")
    }
  }
}

// Test async operations
class AsyncServiceTests {
  println("=== Async Testing ===")
  val service = new AsyncService()
  
  def testFetchData(): Unit = {
    val future = service.fetchData(123)
    val result = Await.result(future, 200.millis)
    if (result == "Data for 123") {
      println("✓ testFetchData: PASSED")
    } else {
      println(s"✗ testFetchData: FAILED (got '$result')")
    }
  }
  
  def testProcessData(): Unit = {
    val future = service.processData("Hello World")
    val result = Await.result(future, 100.millis)
    if (result == 11) {
      println("✓ testProcessData: PASSED")
    } else {
      println(s"✗ testProcessData: FAILED (got $result)")
    }
  }
  
  def testComposition(): Unit = {
    val composed = for {
      data <- service.fetchData(42)
      length <- service.processData(data)
    } yield (data, length)
    
    val (data, length) = Await.result(composed, 300.millis)
    if (data == "Data for 42" && length == data.length) {
      println("✓ testComposition: PASSED")
    } else {
      println(s"✗ testComposition: FAILED (data='$data', length=$length)")
    }
  }
  
  def testErrorHandling(): Unit = {
    // Test multiple times due to randomness
    var successes = 0
    var failures = 0
    
    for (_ <- 1 to 10) {
      try {
        val result = Await.result(service.riskyOperation(), 100.millis)
        successes += 1
      } catch {
        case _: Exception => failures += 1
      }
    }
    
    if (successes > 0 && failures > 0) {
      println(s"✓ testErrorHandling: PASSED ($successes succeeds, $failures fails)")
    } else {
      println("✗ testErrorHandling: FAILED (unexpected behavior)")
    }
  }
  
  def run(): Unit = {
    testFetchData()
    testProcessData()
    testComposition()
    testErrorHandling()
    println()
  }
}

val asyncTests = new AsyncServiceTests()
asyncTests.run()

## 🎲 Property-Based Testing

Testing with generated data to find edge cases.

In [ ]:
// Simulate property-based testing
class PropertyTests {
  println("=== Property-Based Testing ===")
  
  val calculator = new Calculator()
  
// Test that add is commutative: a + b == b + a
  def testAddCommutative(): Unit = {
    var passed = 0
    var failed = 0
    
    for (a <- -10 to 10; b <- -10 to 10) {
      val result1 = calculator.add(a, b)
      val result2 = calculator.add(b, a)
      if (result1 == result2) {
        passed += 1
      } else {
        failed += 1
        println(s"commutative failed: add($a, $b) = $result1 vs add($b, $a) = $result2")
      }
    }
    
    if (failed == 0) {
      println(s"✓ Add commutative: PASSED ($passed cases)")
    } else {
      println(s"✗ Add commutative: FAILED ($failed failures)")
    }
  }
  
// Test that add is associative: (a + b) + c == a + (b + c)
  def testAddAssociative(): Unit = {
    var passed = 0
    var failed = 0
    
    for (a <- -5 to 5; b <- -5 to 5; c <- -5 to 5) {
      val result1 = calculator.add(calculator.add(a, b), c)
      val result2 = calculator.add(a, calculator.add(b, c))
      if (result1 == result2) {
        passed += 1
      } else {
        failed += 1
        println(s"associative failed: ($a+$b)+$c = $result1 vs $a+($b+$c) = $result2")
      }
    }
    
    if (failed == 0) {
      println(s"✓ Add associative: PASSED ($passed cases)")
    } else {
      println(s"✗ Add associative: FAILED ($failed failures)")
    }
  }
  
// Test even/odd properties
  def testEvenOddProperties(): Unit = {
    var evenSquaredEven = 0
    var oddPlusOneEven = 0
    var checked = 0
    
    for (n <- 0 to 20) {
      checked += 1
      
      if (calculator.isEven(n) && calculator.isEven(n * n)) {
        evenSquaredEven += 1
      }
      
      if (!calculator.isEven(n) && calculator.isEven(n + 1)) {
        oddPlusOneEven += 1
      }
    }
    
    println(s"✓ Even squared is even: $evenSquaredEven/$checked cases")
    println(s"✓ Odd + 1 is even: $oddPlusOneEven/$checked cases")
  }
  
  def run(): Unit = {
    testAddCommutative()
    testAddAssociative()
    testEvenOddProperties()
    println()
  }
}

val propertyTests = new PropertyTests()
propertyTests.run()

## 🏗️ TDD Example

Test-driven development: Write tests first, then implement code.

In [ ]:
// TDD Example: Implement a Stack with TDD approach

// First, write the interface and tests (RED step)
trait Stack[T] {
  def push(item: T): Unit
  def pop(): Option[T]
  def peek(): Option[T]
  def isEmpty: Boolean
  def size: Int
}

// Test first (before implementation)
class StackTests {
  println("=== TDD: Stack Tests (RED) ===")
  
  def testEmptyStack(): Unit = {
    val stack = new ArrayStack[Int]()
    if (stack.isEmpty && stack.size == 0 && stack.pop().isEmpty && stack.peek().isEmpty) {
      println("✓ testEmptyStack: PASSED")
    } else {
      println("✗ testEmptyStack: FAILED")
    }
  }
  
  def testPushAndPop(): Unit = {
    val stack = new ArrayStack[Int]()
    stack.push(1)
    stack.push(2)
    stack.push(3)
    
    if (!stack.isEmpty && stack.size == 3) {
      println("✓ Push operations: OK")
    } else {
      println("✗ Push operations: FAILED")
    }
    
    if (stack.peek().contains(3)) {
      println("✓ Peek returns last element")
    } else {
      println("✗ Peek doesn't return last element")
    }
    
    if (stack.pop().contains(3) && stack.size == 2 && stack.pop().contains(2)) {
      println("✓ Pop operations: LIFO order")
    } else {
      println("✗ Pop operations: FAILED")
    }
    
    stack.pop() // remove 1
    if (stack.isEmpty && stack.pop().isEmpty) {
      println("✓ Stack becomes empty after popping all")
    } else {
      println("✗ Stack doesn't become empty")
    }
  }
  
  def run(): Unit = {
    try {
      testEmptyStack()
      testPushAndPop()
      println()
    } catch {
      case ex: Exception => println(s"TDD tests failed: ${ex.getMessage}")
    }
  }
}

// Now implement the Stack (GREEN step)
class ArrayStack[T] extends Stack[T] {
  private var elements = List[T]()
  
  def push(item: T): Unit = {
    elements = item :: elements
  }
  
  def pop(): Option[T] = {
    if (elements.isEmpty) None
    else {
      val result = elements.head
      elements = elements.tail
      Some(result)
    }
  }
  
  def peek(): Option[T] = elements.headOption
  
  def isEmpty: Boolean = elements.isEmpty
  def size: Int = elements.size
}

// Run TDD cycle
val tddTests = new StackTests()
tddTests.run()

println("✓ TDD Cycle Complete: Tests drove implementation!")
println()

## 🏆 Exercises

### Exercise 1: Library Management System Tests

Write comprehensive tests for a library management system.

In [ ]:
// Exercise 1: Library Management System Tests
// FIXME: Replace ??? with your code

// Book and Library classes (from earlier exercise)
case class Book(isbn: String, title: String, author: String)

class Library {
  private var books = Map[String, Book]()
  private var borrowed = Set[String]()
  
  def addBook(book: Book): Unit = {
    books = books + (book.isbn -> book)
  }
  
  def borrowBook(isbn: String): Boolean = {
    if (books.contains(isbn) && !borrowed.contains(isbn)) {
      borrowed = borrowed + isbn
      true
    } else false
  }
  
  def returnBook(isbn: String): Boolean = {
    if (borrowed.contains(isbn)) {
      borrowed = borrowed - isbn
      true
    } else false
  }
  
  def findBook(isbn: String): Option[Book] = books.get(isbn)
  def availableBooks(): List[Book] = books.values.filter(b => !borrowed.contains(b.isbn)).toList
  def borrowedCount(): Int = borrowed.size
  def totalBooks(): Int = books.size
}

// Test suite
class LibraryTestSuite {
  def runAllTests(): Unit = {
    println("Library Management System Test Suite:")
    println("=" * 45)
    
    testEmptyLibrary()
    testAddBook()
    testBorrowAndReturn()
    testBorrowSameBookTwice()
    testReturnUnborrowedBook()
    testAvailableBooks()
    
    println()
  }
  
  def testEmptyLibrary(): Unit = {
    val library = new Library()
    if (library.totalBooks() == 0 && library.borrowedCount() == 0) {
      println("✓ Empty library test: PASSED")
    } else {
      println("✗ Empty library test: FAILED")
    }
  }
  
  def testAddBook(): Unit = {
    val library = new Library()
    val book = Book("123", "Scala Book", "Author")
    library.addBook(book)
    
    if (library.totalBooks() == 1 && library.findBook("123").contains(book)) {
      println("✓ Add book test: PASSED")
    } else {
      println("✗ Add book test: FAILED")
    }
  }
  
  def testBorrowAndReturn(): Unit = {
    ??? // Test borrowing and returning a book
  }
  
  def testBorrowSameBookTwice(): Unit = {
    ??? // Test that same book can't be borrowed twice
  }
  
  def testReturnUnborrowedBook(): Unit = {
    ??? // Test returning a book that wasn't borrowed
  }
  
  def testAvailableBooks(): Unit = {
    ??? // Test available books list
  }
}

// Run the test suite
val libTests = new LibraryTestSuite()
libTests.runAllTests()
println()

### Exercise 2: Async Email Service Tests

Create tests for an asynchronous email service.

In [ ]:
// Exercise 2: Async Email Service Tests
// FIXME: Replace ??? with your code

import scala.concurrent.{Future, Await}
import scala.concurrent.ExecutionContext.Implicits.global
import scala.concurrent.duration._

// Email service
class EmailService {
  def sendEmail(to: String, subject: String): Future[Boolean] = Future {
    // Simulate network operation
    Thread.sleep(50)
    
    // Basic validation
    if (to.contains("@") && subject.nonEmpty) {
      println(s"Email sent to $to with subject '$subject'")
      true
    } else {
      println(s"Email failed for $to")
      false
    }
  }
  
  def sendBulkEmails(recipients: List[String], subject: String): Future[List[String]] = Future {
    Thread.sleep(200)
    recipients.filter(recipient => !recipient.contains("@"))
  }
}

// Test suite for async email service
class EmailServiceTest {
  val service = new EmailService()
  
  def testValidEmail(): Unit = {
    ??? // Test sending valid email
  }
  
  def testInvalidEmail(): Unit = {
    ??? // Test sending invalid email
  }
  
  def testBulkEmails(): Unit = {
    ??? // Test bulk email filtering
  }
  
  def run(): Unit = {
    println("Async Email Service Tests:")
    println("=" * 30)
    
    testValidEmail()
    testInvalidEmail()
    testBulkEmails()
    
    println()
  }
}

// Run async tests
val emailTests = new EmailServiceTest()
emailTests.run()
println()

### Exercise 3: Property-Based Tests

Write property-based tests for mathematical functions.

In [ ]:
// Exercise 3: Property-Based Tests
// FIXME: Replace ??? with your code

// Math functions to test
object MathUtils {
  def factorial(n: Int): Long = {
    if (n <= 1) 1 else n * factorial(n - 1)
  }
  
  def isPrime(n: Int): Boolean = {
    if (n <= 1) false
    else if (n <= 3) true
    else !(2 to math.sqrt(n).toInt).exists(n % _ == 0)
  }
  
  def fibonacci(n: Int): Int = {
    if (n <= 1) n else fibonacci(n - 1) + fibonacci(n - 2)
  }
  
  def gcd(a: Int, b: Int): Int = {
    if (b == 0) a else gcd(b, a % b)
  }
}

// Property-based test suite
class MathPropertyTests {
  def testFactorialProperties(): Unit = {
    ??? // Test factorial(0) = 1, factorial(1) = 1, factorial(n) > factorial(n-1)
  }
  
  def testPrimeProperties(): Unit = {
    ??? // Test 2 is prime, even numbers > 2 not prime, etc.
  }
  
  def testGcdProperties(): Unit = {
    ??? // Test gcd(x, x) = x, gcd(x, 0) = x, etc.
  }
  
  def run(): Unit = {
    println("Math Property Tests:")
    println("=" * 25)
    
    testFactorialProperties()
    testPrimeProperties()
    testGcdProperties()
    
    println()
  }
}

val mathPropertyTests = new MathPropertyTests()
mathPropertyTests.run()
println()

## 📝 What Next?

🎉 **Congratulations!** You've mastered Testing with ScalaTest!

**You've learned:**
- Unit testing with FunSuite
- Assertion and matcher libraries
- Test fixtures and lifecycle management
- Testing asynchronous code (Futures)
- Property-based testing concepts
- Test-driven development (TDD)
- Building comprehensive test suites

**Key Concepts:**
- **Unit tests**: Test individual functions/classes
- **Matchers**: Expressive assertions (`shouldEqual`, `shouldBe`)
- **Fixtures**: Setup cleanup for consistent test state
- **Async testing**: Using `Await` for Future testing
- **Property testing**: Test with generated data
- **TDD cycle**: Red → Green → Refactor

**Next Steps:**
1. Complete exercises - practice all testing techniques
2. Move to **05: Spark Fundamentals**
3. Set up real ScalaTest in a project
4. Explore integration testing and mocking

**Testing Best Practices:**
- Test one thing per test
- Use descriptive test names
- Test normal cases AND edge cases
- Mock external dependencies
- Run tests on every code change

**Advanced Testing:**
- **Mockito** for mocking dependencies
- **Integration tests** across services
- **Load testing** for performance
- **Test coverage** tools (scoverage)

**Continuous Integration:** Set up automated testing pipelines!

---

*"Code without tests is broken by design." - Jacob Kaplan-Moss*